In [1]:
from google.colab import drive 

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [22]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
import pickle
import gzip
from collections import Counter
%matplotlib inline

In [3]:
## 데이터 불러오기
train = pd.read_csv('/content/gdrive/MyDrive/BOAZ/mini/청와대 청원/train.csv', encoding = 'utf-8')
test = pd.read_csv('/content/gdrive/MyDrive/BOAZ/mini/청와대 청원/test.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('/content/gdrive/MyDrive/BOAZ/mini/청와대 청원/sample_submission.csv', encoding = 'utf-8')

In [16]:
## konlpy, mecab 설치
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2023-02-19 17:28:27--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2023-02-19 17:28:27--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4d00df66a7cdebb99dca8af144.dl.dropboxusercontent.com/cd/0/get/B2y4Wk8nD6wnJAPB9XQ195J_I3wUKASCvzDInxrjlwEF75W57DFDFxWKgUR-5VzA6qthH36IcWbYt23Ya-tocbcEktTLs8WKFjSv1muHzDlNZUmxXv1jlHOHdSuIfQkinBWZbQ_DwjjjqG7DfWefKGbNRrlZ_ffzmbrZQDneU9ZM5RrUnhIYw7YfsQXCk5Veegk/file?dl=1# [following]
--2023-02-19 17:28:28--  https://uc4d00df66a7cdebb99dca8af144.dl.dropboxusercontent.com/cd/0/get/B2y4Wk8nD6wn

# 전처리

## 결측치, 불용어 제거(1)

In [5]:
np.sum(train.isnull())

index       0
category    0
data        8
dtype: int64

In [6]:
train.dropna(axis=0, inplace=True)

In [7]:
np.sum(test.isnull())

index    0
data     0
dtype: int64

In [8]:
train['data'] = train['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test['data'] = test['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

# 특수문자 제거
train['data'] = train['data'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\\n\t]+', " ", regex=True)
test['data'] = test['data'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', " ", regex=True)
train['data'] = train['data'].str.replace(r'\t+', " ", regex=True)
test['data'] = test['data'].str.replace(r'\t+', " ", regex=True)
train['data'] = train['data'].str.replace(r'[\\n]+'," ", regex=True)
test['data'] = test['data'].str.replace(r'[\\n]+'," ", regex=True)

<ipython-input-8-709a20f71fbe>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train['data'] = train['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
<ipython-input-8-709a20f71fbe>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  test['data'] = test['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


## 가장 짧은 문장 제거

In [9]:
## 가장 짧은 문장은 삭제(길이=1)
pop_index = list(np.where(train['data'].apply(len)==1)[0])

In [10]:
train.drop(pop_index,inplace=True)
train.reset_index(inplace=True,drop=True)

In [11]:
## 그 다음으로 짧은 문장 삭제(길이=3)
pop_index = list(np.where(train['data'].apply(len)==3)[0])
train.drop(pop_index,inplace=True)
train.reset_index(inplace=True,drop=True)

## 불용어 제거(2)

- 한국어 불용어 100 일부 추가 ([출처](https://bab2min.tistory.com/544))

In [37]:
from konlpy.tag import Mecab

In [45]:
X_train = []
stopwords = ['과', '우리', '나', '좀', '지금', '때', '말', '것', '자', '의', '두', '로', '하나', '명', '살', '건', '를', '부', '중', 
             '고', '위하', '을', '년', '하다', '때문', '저', '들', '데', '게', '개', '도', '또', '일', '경우', '있', '그러나', 
             '더', '되', '한', '그', '그리고', '주', '그런', '정도', '어떻', '시키', '적', '이런', '잘', '같', '에', '오', '이', '걍', 
             '는', '자신', '원', '가', '월', '으로', '아니', '지', '은', '그렇', '보', '않', '씨', '그것', '어떤', '전', '안', '놓', 
             '점', '후', '만', '하', '이렇', '없', '속', '다시', '등', '거', '와', '그러', '싶', '내', '저희', '시간', '제', '수', 
             '알', '분', '번','합니다','입니다','이다','습니다','입니까','습니까','세요','해요','다고','한다고']

mecab = Mecab()

for sentence in train['data'] :
    temp_X = []
    temp_X = mecab.morphs(sentence)

    temp_X = [word for word in temp_X if not word in stopwords]
    X_train.append(temp_X)

In [ ]:
X_test = []
stopwords = ['과', '우리', '나', '좀', '지금', '때', '말', '것', '자', '의', '두', '로', '하나', '명', '살', '건', '를', '부', '중', 
             '고', '위하', '을', '년', '하다', '때문', '저', '들', '데', '게', '개', '도', '또', '일', '경우', '있', '그러나', 
             '더', '되', '한', '그', '그리고', '주', '그런', '정도', '어떻', '시키', '적', '이런', '잘', '같', '에', '오', '이', '걍', 
             '는', '자신', '원', '가', '월', '으로', '아니', '지', '은', '그렇', '보', '않', '씨', '그것', '어떤', '전', '안', '놓', 
             '점', '후', '만', '하', '이렇', '없', '속', '다시', '등', '거', '와', '그러', '싶', '내', '저희', '시간', '제', '수', 
             '알', '분', '번','합니다','입니다','이다','습니다','입니까','습니까','세요','해요','다고','한다고']

mecab = Mecab()

for sentence in test['data'] :
    temp_X = []
    temp_X = mecab.morphs(sentence)

    temp_X = [word for word in temp_X if not word in stopwords]
    X_test.append(temp_X)

## Tokenizer
- 아래는 **baseline code** 그대로 가져온 것입니다

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
vocab_size = 30000
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
# 각 단어별 빈도수 확인
print(tokenizer.word_counts.items())

In [ ]:
max_len = 500
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [ ]:
y_train = to_categorical(train['category'])

## 모델 생성

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 120))
model.add(LSTM(120))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=15)

## 모델 예측

In [ ]:
y_pred = model.predict(X_test)
sample_submission['category'] = y_pred
#sample_submission.to_csv('/content/gdrive/MyDrive/BOAZ/mini/청와대 청원/submission.csv', encoding='utf-8', index = False)